In [1]:
import pandas as pd
import re
import sys
from PyPDF2 import PdfFileReader
import docx2txt
import openpyxl
import gspread
import df2gspread as d2g

In [2]:
#opens the google sheet of pending case notes
    #sets the json to service account path
json_path = gspread.service_account(filename = '/Users/hector/codeup-data-science/293pending_cases/pending_cases.json')
    #opens the google sheet by key found in the address
opens_civil_pending_gs = json_path.open_by_key('1b3fmZrbfwZWMvu4kUGJSSGsp61utlE0Ny-ebozZ5aBk')
    #pulls the data from the google worksheet (civil_pending_notes tab)
civil_pending_notes_tab = opens_civil_pending_gs.get_worksheet(0)
    #puts the data from the google sheet and puts it into a dataframe
civil_pending_notes = pd.DataFrame(civil_pending_notes_tab.get_all_records())

In [3]:
civil_pending_notes

,cause_number,File Date,Plaintiff Name,notes,disposed,dispose_date,On Track,Docket Date,Docket Type
0,1,,,(Look for this case under Plaintiff name G and...,,,,,
1,2,,,Case does not have any information. Please dis...,,,,,
2,6972,,,DISPOSED,TRUE,3/30/2011,,,
3,7038,,,DISPOSED,TRUE,2/11/1999,,,
4,8672,,,DISPOSED,TRUE,11/14/2005,,,
...,...,...,...,...,...,...,...,...,...
1664,99-08-16138-CV,,,DISPOSED,TRUE,,TRUE,,
1665,99-12-16312-CV,,,DISPOSED,TRUE,,TRUE,,
1666,IN THE INTEREST OF-OTH,,,Admin-9 2/23/2001,,,,,
1667,O2-10-18750-MCV,,,Notice of Nonsuit 7/28/2004,,,,,


In [4]:
#Extract pdf text
file = open('/Users/hector/Downloads/CV.PEND.CASE_1-19-2022.pdf', 'rb') #opens pdf
pdfReader = PdfFileReader(file) #reads pdf
count = pdfReader.numPages #counts the number of pages
content = " " #empty variable for the extracted text
for i in range(count): #for loop to extract text from all pages
    page = pdfReader.getPage(i) #gets page numbers
    content += page.extractText() #extracts text from iterated pages
    print(content)#, file = open('yesterdays_causes.txt', 'a')) #prints cause numbers to file. I did this because the extracted texts was seperated into lists by pages.

                 COUNTY OF MAVERICK 293RD DISTRICT COURT CIVIL PENDING CASES AS OF 01/19/2022                                           RAN ON JANUARY 19, 2022 08:09am PAGE: 1        CAUSE #             FILE DATE CAUSE OF ACTION           DOCKT DATE  DOCKET TYPE       ANS FILED  CR CASE #PLAINTIFF NAME                PLAINTIFF ATTORNEY        DEFENDANT NAME                DEFENDANT ATTORNEY        00-02-16392-CV     02/04/2000 (TITLE IV-D OAG USE ONLY                                                   ATTORNEY GENERAL              MELINDA S. LAUCK          CHAVARRIA, JOSE M.                                         CHAVARRIA, AZUCENA                                                                                                 -------------------------------------------------------------------------------------------------------------        1-01-33838-MCV     01/11/2017 MOTOR VEHICLE                                                              CANO, ANA                     ROLANDO SALI

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [13]:
#regex - finds and extracts cause_numbers
finds_cause_numbers = re.findall(r'\d{2}-\d{2}-\d{5}-\w*', str(content))#pattern for cause number
#t_causenum is today's causenumber
pending_cause_number_df = pd.DataFrame(finds_cause_numbers, columns = ['cause_number'])#puts the extract cause number into a df
pending_cause_number_df.shape


(1174, 1)

In [6]:
#adds both lists together in order to search for dups later
df = civil_pending_notes.append(pd.DataFrame(pending_cause_number_df, columns=['cause_number']), ignore_index=True)
df.shape

(2843, 9)

In [7]:
    #drops the duplicated cause numbers and reindexes the dataframe
    #resets the index and drops the output index
    #fills in the na with an empty space to avoid error
df = df.drop_duplicates('cause_number').reset_index(drop=True).fillna(' ')
df.shape


(1677, 9)

In [ ]:
df

In [8]:
total = df.cause_number.count()
total

1677

In [9]:
disposed = (df['disposed']).value_counts()['TRUE']

In [10]:
pending = total - disposed
pending

1383

In [ ]:
#civil_pending_notes_tab.clear()

In [ ]:
#civil_pending_notes_tab.update([df.columns.values.tolist()] + df.values.tolist())

# SCRATCH code

In [ ]:
pending_cause_number_df

In [ ]:
#adds both lists together in order to search for dups later
df = civil_pending_notes.append(pd.DataFrame(pending_cause_number_df, columns=['cause_number']), ignore_index=True)
df

In [ ]:
#drops the duplicated cause numbers
df = df.drop_duplicates('cause_number').fillna(' ').reset_index(drop=True)
df

In [ ]:
# add a sheet with 20 rows and 2 columns
opens_civil_pending_gs.add_worksheet(rows=2000,cols=10,title='new')

# get the instance of the second sheet
civil_pending_notes = opens_civil_pending_gs.get_worksheet(1)

In [ ]:
    #updates the google sheet with the new list of pending cases
civil_pending_notes.update([df.columns.values.tolist()] + df.values.tolist())

In [ ]:
civil_pending_notes

In [ ]:
rangeAll = '{0}!A1:Z'.format( sheetName )
body = {}
resultClear = service.spreadsheets( ).batchUpdate().values( ).clear( spreadsheetId=spreadsheet_id, range=rangeAll,
                                                       body=body ).execute( )

In [ ]:
gc = gspread.service_account(filename = '/Users/hector/Desktop/pau/pending_cases.json')
sht1 = gc.open_by_key('1b3fmZrbfwZWMvu4kUGJSSGsp61utlE0Ny-ebozZ5aBk')


In [ ]:
worksheet = sht1.get_worksheet(0)

In [ ]:

civil_pending_notes = pd.DataFrame(worksheet.get_all_records())


In [ ]:
#civil_pending_notes = pd.read_excel('civil_pending.xlsx')
civil_pending_notes


In [ ]:
df = df.fillna(' ')
df

In [ ]:
worksheet.update([df.columns.values.tolist()] + df.values.tolist())

In [ ]:
#prints to excel
#add notes to this excel file
df.to_excel('pending_cause_numbers.xlsx', sheet_name='pending_cause_numbers')  

#New version of this will be a google sheet

In [ ]:
#opens and reads the previous days pending report i.e. 'yesterdays_pending.pdf'
file = open('yesterdays_pending.pdf', 'rb') #opens pdf
pdfReader = PdfFileReader(file) #reads pdf
count = pdfReader.numPages #counts the number of pages
content = " "
for i in range(count): #for loop to extract text from all pages
    page = pdfReader.getPage(i) #gets page numbers
    content += page.extractText() #extracts text from iterated pages
    print(content)#, file = open('yesterdays_causes.txt', 'a')) #prints cause numbers to file. I did this because the extracted texts was seperated into lists by pages.
    
    

In [ ]:
#extracts cause numbers from the previous day's pending report
y_causenum = re.findall(r'\d{2}-\d{2}-\d{5}-\w*', content)
yesterday_df = pd.DataFrame(y_causenum, columns = ['cause_number'])#puts it into a df
yesterday_df.shape

In [ ]:
#adds both lists together in order to search for dups later
df = todays_df.append(pd.DataFrame(y_causenum, columns=['cause_number']), ignore_index=True)
df.shape #shape of total rows


In [ ]:
#drops the duplicated cause numbers
df = df.drop_duplicates()
df.shape

In [ ]:
df

In [ ]:
#prints to excel
#add notes to this excel file
df.to_excel('pending_causes.xlsx', sheet_name='pending_cause_numbers')  

#New version of this will be a google sheet

In [ ]:
#reads in the most recent excel file that has notes ie 'latest_df'
latest_df = pd.read_excel('pending_causes.xlsx', index_col=0)#index_col = 0 so that it does not add in another indexed col
#this will read in a google sheet that has the notes in it


In [ ]:
latest_df

In [ ]:
#opens and reads the previous days pending report i.e. 'yesterdays_pending.pdf'
file = open('7_pen.pdf', 'rb') #opens pdf
pdfReader = PdfFileReader(file) #reads pdf
count = pdfReader.numPages #counts the number of pages
content = " "
for i in range(count): #for loop to extract text from all pages
    page = pdfReader.getPage(i) #gets page numbers
    content += page.extractText() #extracts text from iterated pages
    print(content)#, file = open('yesterdays_causes.txt', 'a')) #prints cause numbers to file. I did this because the extracted texts was seperated into lists by pages.
    
    

In [ ]:
#extracts new day report cause numbers to be compared to latest report
t_causenum = re.findall(r'\d{2}-\d{2}-\d{5}-\w*', content)
todays_df = pd.DataFrame(t_causenum, columns = ['cause_number'])

In [ ]:
todays_df

In [ ]:
#takes the new day report and adds it to the latest report for comparison
df = latest_df.append(pd.DataFrame(todays_df, columns=['cause_number']), ignore_index=True) #adds both lists together in order to search for dups later
df


In [ ]:
#drops the duplicated cause numbers
df = df.drop_duplicates() 
df
###Needs to be reindexes!!

In [ ]:
#prints back to excel
#update this file
df.to_excel("_pending_causes.xlsx", sheet_name='pending_cause_numbers')  

#New version of this will go back to the google sheet

In [ ]:
#duplicated=df[df.duplicated()] #finds all duplicates
#duplicated.shape #count of rows and columns of the duplicated cause numbers

In [ ]:
#shows ONLY the duplicates
#duplicated.sort_values('cause_number')